In [ ]:
# =================================================================================================
# A. Library Imports and Data Loading
# =================================================================================================
import pandas as pd
import ast

# The file_path must point to the RAW_recipes.csv file.
# This code assumes it is in the same directory as this script.
# If not, please update the path below.
file_path = "/kaggle/RAW_recipes.csv"

recipes_df = None # Initialize recipes_df to None

try:
    # Use the 'python' engine and skip bad lines to handle potential parsing errors
    recipes_df = pd.read_csv(file_path, engine='python', on_bad_lines='skip')
except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")
    print("Please make sure the 'RAW_recipes.csv' file is in the correct directory (e.g., upload it to Colab's session storage).")
    # Assign an empty DataFrame to prevent NameError later
    recipes_df = pd.DataFrame({'name': [], 'ingredients': [], 'steps': []}) # Define with expected columns to avoid errors later
except Exception as e:
    print(f"An unexpected error occurred during file loading: {e}")
    recipes_df = pd.DataFrame({'name': [], 'ingredients': [], 'steps': []})

# Only proceed with preprocessing if recipes_df was successfully loaded and is not empty
if recipes_df is not None and not recipes_df.empty:
    # =================================================================================================
    # B. Data Preprocessing
    # =================================================================================================

    # The 'ingredients' column is stored as a string representation of a list.
    # We need to convert it to an actual Python list.
    # Add error handling for ast.literal_eval
    def safe_literal_eval(x):
        try:
            return ast.literal_eval(x)
        except (ValueError, SyntaxError):
            return [] # Return an empty list or other appropriate value for invalid entries

    recipes_df['ingredients'] = recipes_df['ingredients'].apply(safe_literal_eval)


    # We also convert the list of ingredients to a set for faster lookups.
    recipes_df['ingredients_set'] = recipes_df['ingredients'].apply(lambda x: set(i.lower().strip() for i in x))
else:
    print("Skipping data preprocessing as recipe data was not loaded.")


# =================================================================================================
# C. Core Logic Functions
# =================================================================================================

def suggest_recipes(user_input, num_suggestions=3):
    """
    Suggests the top N recipes based on a user's ingredient list by matching.

    Args:
        user_input (str): A comma-separated string of ingredients.
        num_suggestions (int): The number of recipes to suggest.

    Returns:
        list: A list of dictionaries, each containing a recipe match.
    """
    # Check if recipes_df is available and not empty
    if recipes_df is None or recipes_df.empty:
        return []

    # Create a set of the user's ingredients for fast lookups.
    user_ingredients_set = set(i.strip().lower() for i in user_input.split(','))

    # This list will store all the matching recipes and their scores.
    matches = []

    # Loop through each recipe in the entire DataFrame.
    for index, row in recipes_df.iterrows():
        # Find the number of ingredients that overlap between the user and the recipe.
        matched_ingredients = user_ingredients_set.intersection(row['ingredients_set'])
        match_score = len(matched_ingredients)

        # If there's at least one matching ingredient, add it to our list.
        if match_score > 0:
            recipe_ingredients_set = row['ingredients_set']
            missing_ingredients_list = list(recipe_ingredients_set - user_ingredients_set)

            matches.append({
                'title': row['name'],
                'instructions': row['steps'],
                'original_ingredients': row['ingredients'],
                'match_score': match_score,
                'missing_ingredients': missing_ingredients_list
            })

    # Sort the matches by score in descending order (highest score first).
    matches.sort(key=lambda x: x['match_score'], reverse=True)

    # Return only the top N suggestions.
    return matches[:num_suggestions]


# =================================================================================================
# D. User Interface and Execution
# =================================================================================================

def main():
    """
    Main function to run the recipe generator.
    """
    print("Welcome to the Basic Recipe Generator!")
    print("This tool suggests recipes based on the ingredients you have on hand.")

    # Get user input.
    user_input = input("\nEnter your ingredients (comma separated, e.g., chicken, onion, tomato): ")

    # Get recipe suggestions based on the user's input.
    suggested_recipes = suggest_recipes(user_input, num_suggestions=3)

    # Print the results.
    print("\n--- Here are some recipe suggestions for you: ---")
    if not suggested_recipes:
        print("No recipes found for your ingredients. Please try different ingredients or ensure the data file was loaded correctly.")
    else:
        for recipe in suggested_recipes:
            print(f"\n--- {recipe['title'].upper()} ---")
            print(f"Match Score: {recipe['match_score']} ingredients matched.")
            print("\nIngredients:")
            print(", ".join(recipe['original_ingredients']))

            print("\nInstructions:")
            print(recipe['instructions'])

            if recipe['missing_ingredients']:
                print("\n** You are missing the following ingredients:", ", ".join(recipe['missing_ingredients']), "**")
            else:
                print("\n** You have all the ingredients! **")

if __name__ == "__main__":
    main()

Error: The file '/kaggle/RAW_recipes.csv' was not found.
Please make sure the 'RAW_recipes.csv' file is in the correct directory (e.g., upload it to Colab's session storage).
Skipping data preprocessing as recipe data was not loaded.
Welcome to the Basic Recipe Generator!
This tool suggests recipes based on the ingredients you have on hand.
